In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch
!pip install numpy
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=39b2f29db5c1ce53a8f8c0d590da3d43fc04958e675212ebde74d7550e94ab6b
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import torchvision.models as models
import time
import datetime
from efficientnet_pytorch import EfficientNet

In [ ]:
# 모델 학습에 사용할 연산 장치를 설정합니다..
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')

print(DEVICE)

cuda


In [ ]:
# 모델학습에 사용할 하이퍼 파라미터를 설정합니다.
batch_size = 32
epochs = 5

In [ ]:
path = '/content/drive/MyDrive/data/ComputerVision'

In [ ]:
# Fashion MNIST Train 데이터를 받아옵니다.
train_dataset = datasets.FashionMNIST(root=path, train = True, download = True, transform = transforms.ToTensor())
# Fashion MNIST Test 데이터를 받아옵니다.
test_dataset = datasets.FashionMNIST(root=path, train = False, download = True, transform = transforms.ToTensor())

In [ ]:
# 받아온 MNIST데이터를 DataLoader를 이용해 batch_size 차원을 추가해서 변환합니다.
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size= batch_size, shuffle=True)

In [ ]:
for x, y in train_loader:
  print('변환 후 데이터 Shape : ', x.size())
  print('레이블 데이터 수 : ', y.size())
  break

변환 후 데이터 Shape :  torch.Size([32, 1, 28, 28])
레이블 데이터 수 :  torch.Size([32])


In [ ]:
# CNN 모델을 정의합니다.
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    # 첫번째 Convolution Layer를 추가합니다.
    # 위에서 확인했듯이 이미지 파일의 channel이 1이므로 처음 입력값에 동일하게 1로 설정합니다
    # 나머지 파라미터는 적절한 값을 주어 구조를 설계합니다.
    self.conv1 = nn.Conv2d(
        in_channels = 1,
        out_channels = 8,
        kernel_size = 3,
        padding = 1
    )

    # 두번째 Convolution Layer를 추가합니다.
    self.conv2 = nn.Conv2d(
        in_channels = 8,
        out_channels = 16,
        kernel_size = 3,
        padding = 1
    )

    # 주어진 데이터의 Feature size를 줄여주는 ooling Layer를 추가합니다. 
    self.pool = nn.MaxPool2d(
        kernel_size = 2
    )

    # 이미지 분류를 위한 Fully Connected Layer를 추가합니다
    self.fc1 = nn.Linear(28 * 28, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 32)
    self.fc4 = nn.Linear(32, 10)
    

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)

    # 이미지 데이터를 Fully Connected Layer의 Input으로 넣기 위해 1차원으로 Flatten 시켜줍니다.
    x = x.view(-1, 28 * 28)

    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.log_softmax(x)
    return x

In [ ]:
# GPU에 모델을 할당합니다.
model = CNN().to(DEVICE)

# 학습에 사용할 최적화 함수와 학숩률을 설정합니다.
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# 학습에 사용할 비용함수를 정의합니다.
criterion = nn.CrossEntropyLoss()

print(model)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=10, bias=True)
)


In [ ]:
# 모델을 학습할 때 사용할 함수를 정의합니다.
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [ ]:
# 모델을 평가할 때 사용할 함수를 정의합니다.
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
start = time.time()
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))
    
sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.322237
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.677294
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.891655
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.653898
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.655476
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.518379
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.449814
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.551263
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.424114
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.392018

[EPOCH: 1], 	Test Loss: 0.0141, 	Test Accuracy: 83.14 % 

Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.388195
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.491798
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.406942
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.252935
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.241703
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.171356
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.481676
T

In [ ]:
# torchvision에서 Resnet 모델을 불러옵니다.
model = models.resnet34(pretrained = False)

# 현재 학습하고자 하는 데이터에 적합하게 channel수를 1로 변경합니다.
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)

# Fully Connected Layer의 출력층 개수를 데이터에 맞게 수정합니다.
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# 모델을 연산장치에 할당합니다.
model = model.cuda()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

print(model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
start = time.time()
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))
    
sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)    

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.626826
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.619241
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.620139
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.477209
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.310428
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.494821
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.487029
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.392368
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.328003
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.235117

[EPOCH: 1], 	Test Loss: 0.0121, 	Test Accuracy: 86.40 % 

Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.292259
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.294077
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.475642
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.135433
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.490600
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.401930
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.310215
T

In [ ]:
# EfficientNet-B3 모델을 불러옵니다.
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=10)

# 입력층 Conv Layer의 channel 값을 1로 수정합니다.
model._conv_stem = nn.Conv2d(1, 40, kernel_size=7, stride=2, padding=3,bias=False)
model = model.cuda()

Loaded pretrained weights for efficientnet-b3


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

print(model)

EfficientNet(
  (_conv_stem): Conv2d(1, 40, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
 

In [ ]:
start = time.time()
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))
    
sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)    

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.425670
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.596149
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.669461
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.435456
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.277851
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.588510
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.615530
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.490416
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.473814
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.307307

[EPOCH: 1], 	Test Loss: 0.0116, 	Test Accuracy: 86.95 % 

Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.817764
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 1.100895
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.279844
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.512571
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.337717
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.362043
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.700994
T